In [1]:
; cd ..

/Users/rhenriquez/.julia/dev/LITS


In [2]:
] activate

Activating environment at `~/.julia/dev/LITS/Project.toml`


In [3]:
] instantiate

In [4]:
using Revise
using LITS
import PowerSystems
using Sundials
const PSY = PowerSystems

PowerSystems

## Structs

In [5]:
"""
Parameters of an inner loop controller using a PI voltage controller and a PI current controller.

#Arguments
- `ω0::Float64`: Reference frequency in per unit.
"""

mutable struct NoPLL <: PSY.FrequencyEstimator
    ω0::Float64
    n_states::Int64
    states::Vector{Symbol}
    ext::Dict{String, Any}

    function NoPLL(ω0, ext=Dict{String, Any}(), )
        new(ω0, 0, Vector{Symbol}(), ext)
    end

end

get_ω0(value::NoPLL) = value.ω0
PSY.get_states(value::NoPLL) = value.states
PSY.get_n_states(value::NoPLL) = value.n_states
PSY.get_ext(value::NoPLL) = value.ext;

In [6]:
"""
Parameters of an inner loop controller using a PI voltage controller and a PI current controller.

#Arguments
- `kpv::Float64`: voltage controller proportional gain
- `kiv::Float64`: voltage controller integral gain
- `kffi::Float64`: Binary variable to enable feed-forward gain of current
- `kpc::Float64`: current controller proportional gain, validation range
- `kic::Float64`: current controller integral gain, validation range
- `kffv::Float64`: Binary variable to enable feed-forward gain of voltage
"""

mutable struct PI_VoltageCurrent <: PSY.VSControl
    kpv::Float64
    kiv::Float64
    kffi::Float64
    kpc::Float64
    kic::Float64
    kffv::Float64
    n_states::Int64
    states::Vector{Symbol}
    ext::Dict{String, Any}

    function PI_VoltageCurrent(kpv, kiv, kffi, kpc, kic, kffv, ext=Dict{String, Any}(), )

        states = [:ξ_d, :ξ_q, :γ_d, :γ_q]
        n_states = 4

        new(kpv, kiv, kffi, kpc, kic, kffv, n_states, states, ext)
    end
end

PSY.get_kpv(value::PI_VoltageCurrent) = value.kpv
PSY.get_kiv(value::PI_VoltageCurrent) = value.kiv
PSY.get_kffi(value::PI_VoltageCurrent) = value.kffi
PSY.get_kpc(value::PI_VoltageCurrent) = value.kpc
PSY.get_kic(value::PI_VoltageCurrent) = value.kic
PSY.get_kffv(value::PI_VoltageCurrent) = value.kffv
PSY.get_states(value::PI_VoltageCurrent) = value.states
PSY.get_n_states(value::PI_VoltageCurrent) = value.n_states
PSY.get_ext(value::PI_VoltageCurrent) = value.ext;

In [7]:
"""
Parameters of an Active Power Droop Controller

# Arguments
- `d_ω::Float64`: Frequency-Power droop gain
"""

mutable struct PowerDroop <: PSY.ActivePowerControl
    d_ω::Float64
    n_states::Int64
    states::Vector{Symbol}
    ext::Dict{String, Any}
    
    function PowerDroop(d_ω, ext=Dict{String, Any}(),)
        
        new(d_ω, 1, [:δθ_vsm], ext) #TODO: I Used δθ_vsm for port issues. Replace with proper symbol. 
    end
end

get_d_ω(value::PowerDroop) = value.d_ω
PSY.get_states(value::PowerDroop) = value.states
PSY.get_n_states(value::PowerDroop) = value.n_states
PSY.get_ext(value::PowerDroop) = value.ext;

"""
Parameters of a Virtual PI AVR.

# Arguments
- `kp_avr::Float64`: Proportional gain for AVR
- `ki_avr::Float64`: Integral gain for AVR
"""

mutable struct Virtual_PI_AVR <: PSY.ReactivePowerControl
    kp_avr::Float64
    ki_avr::Float64
    n_states::Int64
    states::Vector{Symbol}
    ext::Dict{String, Any}
    
    function Virtual_PI_AVR(kp_avr, ki_avr, ext=Dict{String, Any}(), )
        
        new(kp_avr, ki_avr, 1, [:x_avr], ext)
    end
end

get_kp_avr(value::Virtual_PI_AVR) = value.kp_avr
get_ki_avr(value::Virtual_PI_AVR) = value.ki_avr
PSY.get_states(value::Virtual_PI_AVR) = value.states
PSY.get_n_states(value::Virtual_PI_AVR) = value.n_states
PSY.get_ext(value::Virtual_PI_AVR) = value.ext;

In [8]:
## Inverter components
nopll  = NoPLL(1.0)
innerloop = PI_VoltageCurrent(
    0.59, #kpv
    232.0, #kiv
    0.0, #kffi
    0.73, #kpc
    0.0059, #kic
    0.0) #kffv
pdroop = PowerDroop(2.0) #d_ω
virtual_avr = Virtual_PI_AVR(
    0.001, #kp_avr
    0.05) #ki_avr
outerloop = PSY.VirtualInertiaQdroop(pdroop, virtual_avr)
filter_test = PSY.LCLFilter(
    0.08, #Series inductance lf in pu
    0.003, #Series resitance rf in pu
    0.074, #Shunt capacitance cf in pu
    0.2, #Series ractance rg to grid connection (#Step up transformer or similar)
    0.01,
) #Series resistance lg to grid connection (#Step up transformer or similar)
dc_source = PSY.FixedDCSource(2440.0)
conv_droop = PSY.AvgCnvFixedDC(
    2440.0, #Rated Voltage
    100.0,
) #Rated MVA

PowerSystems.AvgCnvFixedDC:
   v_rated: 2440.0
   s_rated: 100.0
   ext: Dict{String,Any}()
   states: Symbol[]
   n_states: 0

In [9]:
## System
nodes = [
    PSY.Bus(
        1, #number
        "Bus 1", #Name
        "REF", #BusType (REF, PV, PQ)
        0, #Angle in radians
        1.05, #Voltage in pu
        (min = 0.94, max = 1.06), #Voltage limits in pu
        69, #Base voltage in kV
        nothing,
        nothing,
    ),
    PSY.Bus(2, "Bus 2", "PV", 0, 1.0, (min = 0.94, max = 1.06), 69, nothing, nothing),
]

2-element Array{PowerSystems.Bus,1}:
 PowerSystems.Bus(1, "Bus 1", PowerSystems.BusTypes.REF, 0.0, 1.05, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("7b9225a3-63b3-4ffa-9ff9-93608988158e"), nothing))
 PowerSystems.Bus(2, "Bus 2", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("0f986873-2cb4-445a-8f77-8a498a126e32"), nothing))  

In [10]:
branches = [PSY.Line(
    "Line1", #name
    true, #available
    0.0, #active power flow initial condition (from-to)
    0.0, #reactive power flow initial condition (from-to)
    PSY.Arc(from = nodes[1], to = nodes[2]), #Connection between buses
    0.01, #resistance in pu
    0.05, #reactance in pu
    (from = 0.0, to = 0.0), #susceptance in pu
    18.046, #rate in MW
    1.04,
)]  #angle limits (-min and max)

1-element Array{PowerSystems.Line,1}:
 PowerSystems.Line("Line1", true, 0.0, 0.0, PowerSystems.Arc(PowerSystems.Bus(1, "Bus 1", PowerSystems.BusTypes.REF, 0.0, 1.05, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("7b9225a3-63b3-4ffa-9ff9-93608988158e"), nothing)), PowerSystems.Bus(2, "Bus 2", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("0f986873-2cb4-445a-8f77-8a498a126e32"), nothing)), InfrastructureSystems.InfrastructureSystemsInternal(UUID("83eb0560-00cd-456c-b5c1-020d5ccd5e97"), nothing)), 0.01, 0.05, (from = 0.0, to = 0.0), 18.046, (min = -1.04, max = 1.04), PowerSystems.Service[], Dict{String,Any}(), InfrastructureSystems.Forecasts(Dict{InfrastructureSystems.ForecastKey,InfrastructureSystems.ForecastInternal}(), nothing), InfrastructureSystems.InfrastructureSystemsInternal(UUID("e25a8

In [11]:
inf_source = PSY.Source(
    "InfBus", #name
    true, #availability
    nodes[1], #bus
    1.05, #VR
    0.0, #VI
    0.000005,
) #Xth

InfBus (PowerSystems.Source):
   name: InfBus
   available: true
   bus: PowerSystems.Bus(1, "Bus 1", PowerSystems.BusTypes.REF, 0.0, 1.05, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("7b9225a3-63b3-4ffa-9ff9-93608988158e"), nothing))
   V_R: 1.05
   V_I: 0.0
   X_th: 5.0e-6
   services: PowerSystems.Service[]
   ext: Dict{String,Any}()

In [12]:
inv_droop = PSY.DynamicInverter(
        1, #Number
        "inv_droop", #name
        nodes[2], #bus
        1.0, # ω_ref,
        1.00, #V_ref
        0.5, #P_ref
        0.0, #Q_ref
        100.0, #MVABase
        conv_droop, #converter
        outerloop, #outer control
        innerloop, #inner control voltage source
        dc_source, #dc source
        nopll, #pll
        filter_test,
    ) #filter

inv_droop (PowerSystems.DynamicInverter{PowerSystems.AvgCnvFixedDC,PowerSystems.VirtualInertiaQdroop{PowerDroop,Virtual_PI_AVR},PI_VoltageCurrent,PowerSystems.FixedDCSource,NoPLL,PowerSystems.LCLFilter}):
   number: 1
   name: inv_droop
   bus: PowerSystems.Bus(2, "Bus 2", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("0f986873-2cb4-445a-8f77-8a498a126e32"), nothing))
   ω_ref: 1.0
   V_ref: 1.0
   P_ref: 0.5
   Q_ref: 0.0
   MVABase: 100.0
   converter: PowerSystems.AvgCnvFixedDC(2440.0, 100.0, Dict{String,Any}(), Symbol[], 0, InfrastructureSystems.InfrastructureSystemsInternal(UUID("9f0f8160-a4f9-4489-90a0-bb7985449e68"), nothing))
   outercontrol: PowerSystems.VirtualInertiaQdroop{PowerDroop,Virtual_PI_AVR}(PowerDroop(2.0, 1, Symbol[:δθ_vsm], Dict{String,Any}()), Virtual_PI_AVR(0.001, 0.0001, 1, Symbol[:x_avr], Dict{String,Any}()), Dict{String,Any}(), Symbol[:δθ_vsm, 

## Models

In [13]:
##NoPLL
function LITS.mdl_freq_estimator_ode!(
    device_states,
    output_ode,
    f0,
    ω_sys,
    device::PSY.DynamicInverter{C, O, VC, DC, NoPLL, F},
) where {
    C <: PSY.Converter,
    O <: PSY.OuterControl,
    VC <: PSY.VSControl,
    DC <: PSY.DCSource,
    F <: PSY.Filter,
}
    pll_control = PSY.get_freq_estimator(device)
    ω0 = get_ω0(pll_control)
    
end

In [27]:
## Inner Control
function LITS.mdl_VScontrol_ode!(
    device_states,
    output_ode,
    device::PSY.DynamicInverter{C, O, PI_VoltageCurrent, DC, P, F},
) where {
    C <: PSY.Converter,
    O <: PSY.OuterControl,
    DC <: PSY.DCSource,
    P <: PSY.FrequencyEstimator,
    F <: PSY.Filter,
}
    #Obtain external states for component
    external_ix = LITS.get_input_port_ix(device, PI_VoltageCurrent)
    Id_o = device_states[external_ix[1]]
    Iq_o = device_states[external_ix[2]] 
    Id_cnv = device_states[external_ix[3]]
    Iq_cnv = device_states[external_ix[4]]
    Vd_o = device_states[external_ix[5]]
    Vq_o = device_states[external_ix[6]]
    
    #Obtain inner variables for component
    ω_olc = LITS.get_inner_vars(device)[LITS.ω_control_var]
    V_olc_ref = LITS.get_inner_vars(device)[LITS.v_control_var]
    Vdc = LITS.get_inner_vars(device)[LITS.Vdc_var]
    
    #Get Voltage Controller parameters
    vscontrol = PSY.get_vscontrol(device)
    filter = PSY.get_filter(device)
    kpv = PSY.get_kpv(vscontrol)
    kiv = PSY.get_kiv(vscontrol)
    kffv = PSY.get_kffv(vscontrol)
    cf = PSY.get_cf(filter)
    
    #Get Current Controller Parameters
    kpc = PSY.get_kpc(vscontrol)
    kic = PSY.get_kic(vscontrol)
    kffi = PSY.get_kffi(vscontrol)
    lf = PSY.get_lf(filter)
    #rf = PSY.get_rf(filter) We can generalize if the filter have R or not.
    
    #Obtain indices for component w/r to device
    local_ix = LITS.get_local_state_ix(device, PI_VoltageCurrent)
    
    #Define internal states for inner control
    internal_states = @view device_states[local_ix]
    ξ_d = internal_states[1]
    ξ_q = internal_states[2]
    γ_d = internal_states[3]
    γ_q = internal_states[4]
    
    #Inputs to Voltage Controller (control signals)
    Vd_ref = V_olc_ref #Create Inner var for references in both d and q.
    Vq_ref = 0 #TODO: Create Inner var for references in both d and q.
    
    #Voltage Control ODES
    #PI Integrator (internal state)
    output_ode[local_ix[1]] = (Vd_ref - Vd_o)
    output_ode[local_ix[2]] = (Vq_ref - Vq_o)
    
    #Inputs to Current Controller (control signals)
    Id_vc_ref = kpv * (Vd_ref - Vd_o) + kiv * ξ_d + kffi * (Id_o - cf * ω_olc * Vq_o) 
    Iq_vc_ref = kpv * (Vq_ref - Vq_o) + kiv * ξ_q + kffi * (Iq_o + cf * ω_olc * Vd_o)
    
    #Current Control ODEs
    #PI Integrator (internal state)
    output_ode[local_ix[3]] = Id_vc_ref - Id_cnv
    output_ode[local_ix[4]] = Iq_vc_ref - Iq_cnv
    
    #Voltage reference to converter
    Vd_cc_ref = kpc * (Id_vc_ref - Id_cnv) + kic * γ_d + kffv * (Vd_o - lf * ω_olc * Iq_cnv)
    Vq_cc_ref = kpc * (Iq_vc_ref - Iq_cnv) + kic * γ_q + kffv * (Vq_o + lf * ω_olc * Id_cnv)
    
    #Update inner_vars
    LITS.get_inner_vars(device)[LITS.md_var] = Vd_cc_ref / Vdc
    LITS.get_inner_vars(device)[LITS.mq_var] = Vq_cc_ref / Vdc
end

In [40]:
## Outer Loop
function LITS.mdl_outer_ode!(
    device_states,
    output_ode,
    f0,
    ω_sys,
    device::PSY.DynamicInverter{
        C,
        PSY.VirtualInertiaQdroop{PowerDroop, Virtual_PI_AVR},
        VC,
        DC,
        P,
        F,
    },
) where {
    C <: PSY.Converter,
    VC <: PSY.VSControl,
    DC <: PSY.DCSource,
    P <: PSY.FrequencyEstimator,
    F <: PSY.Filter,
}
    
    #Obtain external states inputs for component
    external_ix = LITS.get_input_port_ix(
        device,
        PSY.VirtualInertiaQdroop{PowerDroop, Virtual_PI_AVR},
    )   
    Vd_o = device_states[external_ix[1]]
    Vq_o = device_states[external_ix[2]]
    Id_o = device_states[external_ix[3]]
    Iq_o = device_states[external_ix[4]]
    #Problem when some states are not defined? Since Outer has a lot of state_ports that may not exist.
    
    #Get Active Power Controller parameters
    outer_control = PSY.get_outercontrol(device)
    active_power_control = PSY.get_active_power(outer_control)
    d_ω = get_d_ω(active_power_control)
    
    #Get Reactive Power Controller parameters
    reactive_power_control = PSY.get_reactive_power(outer_control)
    kp_avr = get_kp_avr(reactive_power_control)
    ki_avr = get_ki_avr(reactive_power_control)
    
    #Obtain external parameters
    p_ref = PSY.get_ext(device)[LITS.CONTROL_REFS][LITS.P_ref_index]
    ω_ref = PSY.get_ext(device)[LITS.CONTROL_REFS][LITS.ω_ref_index]
    V_ref = PSY.get_ext(device)[LITS.CONTROL_REFS][LITS.V_ref_index]
    
    #Obtain inner vars
    V_tR = LITS.get_inner_vars(device)[LITS.VR_inv_var]
    V_tI = LITS.get_inner_vars(device)[LITS.VI_inv_var]
    
    #Compute electric variables
    Pe = (Vd_o * Id_o + Vq_o * Iq_o) #No Resistance in output.
    #Vo_mag = sqrt(V_tR^2 + V_tI^2)
    Vo_mag = sqrt(Vd_o^2 + Vq_o^2)
    ω_olc = ω_ref + d_ω * (p_ref - Pe)
    
    #Obtain indices for component w/r to device
    local_ix = LITS.get_local_state_ix(
        device,
        PSY.VirtualInertiaQdroop{PowerDroop, Virtual_PI_AVR},
    )
    
    #Define internal states for outer loop control
    internal_states = @view device_states[local_ix]
    θ_olc = internal_states[1]
    x_avr = internal_states[2]
    
    #Compute ODEs for Active Power Droop
    output_ode[local_ix[1]] = 2 * π * f0 * (ω_olc - ω_sys)
    
    #Compute ODEs for Virtual AVR
    output_ode[local_ix[2]] = V_ref - Vo_mag
    
    #Update Inner Vars
    LITS.get_inner_vars(device)[LITS.ω_control_var] = ω_olc
    LITS.get_inner_vars(device)[LITS.v_control_var] = kp_avr * (V_ref - Vo_mag) + ki_avr * x_avr
end

In [16]:
sys = PSY.System(100.0, frequency = 50.0)

""
""


In [17]:
for bus in nodes
    PSY.add_component!(sys, bus)
end

In [18]:
for lines in branches
    PSY.add_component!(sys, lines)
end

In [19]:
PSY.add_component!(sys, inf_source)

In [20]:
PSY.add_component!(sys, inv_droop)

┌ Warning: struct DynamicInverter does not exist in validation configuration file, validation skipped
└ @ InfrastructureSystems /Users/rhenriquez/.julia/packages/InfrastructureSystems/qvTbP/src/validation.jl:42


In [21]:
sys

,ConcreteType,SuperTypes
,String,String
1,Bus,Topology <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
2,"DynamicInverter{PowerSystems.AvgCnvFixedDC,PowerSystems.VirtualInertiaQdroop{PowerDroop,Virtual_PI_AVR},PI_VoltageCurrent,PowerSystems.FixedDCSource,NoPLL,PowerSystems.LCLFilter}",DynamicInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
3,Line,ACBranch <: Branch <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any
4,Source,StaticInjection <: Device <: Component <: PowerSystemType <: InfrastructureSystemsType <: Any


In [22]:
branches_fault = [PSY.Line(
    "Line1", #name
    true, #available
    0.0, #active power flow initial condition (from-to)
    0.0, #reactive power flow initial condition (from-to)
    PSY.Arc(from = nodes[1], to = nodes[2]), #Connection between buses
    0.02, #resistance in pu
    0.1, #reactance in pu
    (from = 0.0, to = 0.0), #susceptance in pu
    18.046, #rate in MW
    1.04,
)]

1-element Array{PowerSystems.Line,1}:
 PowerSystems.Line("Line1", true, 0.0, 0.0, PowerSystems.Arc(PowerSystems.Bus(1, "Bus 1", PowerSystems.BusTypes.REF, 0.0, 1.05, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("7b9225a3-63b3-4ffa-9ff9-93608988158e"), nothing)), PowerSystems.Bus(2, "Bus 2", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("0f986873-2cb4-445a-8f77-8a498a126e32"), nothing)), InfrastructureSystems.InfrastructureSystemsInternal(UUID("e5f66a07-e51a-48d8-a145-f4e476dc1248"), nothing)), 0.02, 0.1, (from = 0.0, to = 0.0), 18.046, (min = -1.04, max = 1.04), PowerSystems.Service[], Dict{String,Any}(), InfrastructureSystems.Forecasts(Dict{InfrastructureSystems.ForecastKey,InfrastructureSystems.ForecastInternal}(), nothing), InfrastructureSystems.InfrastructureSystemsInternal(UUID("1aa7f1

In [23]:
inv_droop

inv_droop (PowerSystems.DynamicInverter{PowerSystems.AvgCnvFixedDC,PowerSystems.VirtualInertiaQdroop{PowerDroop,Virtual_PI_AVR},PI_VoltageCurrent,PowerSystems.FixedDCSource,NoPLL,PowerSystems.LCLFilter}):
   number: 1
   name: inv_droop
   bus: PowerSystems.Bus(2, "Bus 2", PowerSystems.BusTypes.PV, 0.0, 1.0, (min = 0.94, max = 1.06), 69.0, nothing, nothing, Dict{String,Any}(), InfrastructureSystems.InfrastructureSystemsInternal(UUID("0f986873-2cb4-445a-8f77-8a498a126e32"), nothing))
   ω_ref: 1.0
   V_ref: 1.0
   P_ref: 0.5
   Q_ref: 0.0
   MVABase: 100.0
   converter: PowerSystems.AvgCnvFixedDC(2440.0, 100.0, Dict{String,Any}(), Symbol[], 0, InfrastructureSystems.InfrastructureSystemsInternal(UUID("9f0f8160-a4f9-4489-90a0-bb7985449e68"), nothing))
   outercontrol: PowerSystems.VirtualInertiaQdroop{PowerDroop,Virtual_PI_AVR}(PowerDroop(2.0, 1, Symbol[:δθ_vsm], Dict{String,Any}()), Virtual_PI_AVR(0.001, 0.0001, 1, Symbol[:x_avr], Dict{String,Any}()), Dict{String,Any}(), Symbol[:δθ_vsm, 

In [24]:
Ybus_fault = PSY.Ybus(branches_fault, nodes)[:,:]
Ybus_change = ThreePhaseFault(
    1.0, #change at t = 1.0
    Ybus_fault,
) #New YBus

ThreePhaseFault(1.0, 
  [1, 1]  =  1.92308-9.61538im
  [2, 1]  =  -1.92308+9.61538im
  [1, 2]  =  -1.92308+9.61538im
  [2, 2]  =  1.92308-9.61538im)

In [25]:
tspan = (0.0, 30.0)
x0_guess = [
    1.0, #V1_R
    1.0, #V2_R
    0.0, #V1_I
    0.0, #V2_I
    0.1, #θ
    0.0, #x_avr
    0.0015, #ξ_d 
    -0.08, #ξ_q
    0.05, #γ_d
    -0.001, #γ_q
    0.5, #id_cnv
    0.0, #iq_cnv
    0.95, #Vd_o
    -0.1, #Vq_o
    0.49, #Id_o
    -0.1, #Iq_o
    ]

16-element Array{Float64,1}:
  1.0   
  1.0   
  0.0   
  0.0   
  0.1   
  0.0   
  0.0015
 -0.08  
  0.05  
 -0.001 
  0.5   
  0.0   
  0.95  
 -0.1   
  0.49  
 -0.1   

In [41]:
sim = LITS.Simulation(sys, tspan, Ybus_change, initial_guess = x0_guess)

┌ Info: Initializing Simulation States
└ @ LITS /Users/rhenriquez/.julia/dev/LITS/src/base/simulation.jl:32


Simulation()


In [42]:
sim.x0_init

16-element Array{Float64,1}:
     1.0499970899127102   
     0.9782601390584762   
    -2.1318038978137983e-5
    -0.20738153324076636  
    -0.1981271044960856   
 18615.797086923147       
     0.0011577115950125103
    -0.01791811290260903  
   372.02468668387587     
     1.5281560378332726   
     0.2685890900429024   
    -4.157002193405295    
     1.8615797086923147   
    -7.493879100095944e-29
     0.2685890900429024   
    -4.294759091848526    

In [43]:
LITS.print_init_states(sim)

Differential States
inv_droop
iq_o -4.294759091848526
δθ_vsm -0.1981271044960856
id_o 0.2685890900429024
ξ_q -0.01791811290260903
x_avr 18615.797086923147
vd_cap 1.8615797086923147
ξ_d 0.0011577115950125103
iq_c -4.157002193405295
γ_q 1.5281560378332726
γ_d 372.02468668387587
id_c 0.2685890900429024
vq_cap -7.493879100095944e-29


In [44]:
small_sig = small_signal_analysis(sim)

The system is not small signal stable
